<a href="https://colab.research.google.com/github/SARA3SAEED/DA-Mu/blob/main/s09b_databases_importing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADVANCED PANDAS: DATA IMPORTING & WEB SCRAPING

Course Outline:
- Basic Data Importing
    - Flat Files (.csv, .tsv, .txt)
    - Excel Files (.xlsx)
    - Other Files (.dta, .mat, .. etc)
    - Basic Data Importing Exercises
- ***Importing Data from Databases***
    - ***SQL Crash Course***
    - ***Database Files (.db, .sqlite, .. etc)***
    - ***Case-study: Chinook Database***
- Importing Data from the Internet
    - HTML & CSS Crash Course
    - Web Scraping Basics
    - Working with JSON Data & APIs
    - Case-study: Wuzzuf.com [Web Scraping]

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

==========

## Importing Data from Databases

- Relational Databases Basics
    - Spreadsheets vs Databases
        - Excel vs. Access (Demo)
    - Common Relational Databases
- SQL Crash Course
- Importing Database Files in Pandas

### SQL Crash Course

- SQL Tutorial:
    - https://www.w3schools.com/sql/
    - https://www.sololearn.com/learning/1060


- SQL Online IDEs:
    - https://sqliteonline.com/
    - http://sqlfiddle.com/

#### Pandas vs SQL (Demo)

In [ ]:
df = pd.read_excel('data/students.xlsx')
df

In [ ]:
df.info()

###### *selecting the column 'first_name' from students table*

In [ ]:
# SQL Syntax
''' SELECT first_name
FROM students
'''

In [ ]:
# Pandas Syntax
df['first_name']

###### selecting all columns from students table

In [ ]:
# SQL Syntax
''' SELECT *
FROM students
'''

In [ ]:
# Pandas Syntax
df

###### let's select all Egyptian students

In [ ]:
# SQL Syntax
''' SELECT *
FROM students
'''

In [ ]:
# Pandas Syntax
df

###### selecting all names which start with 'M' letter

In [ ]:
# SQL Syntax
''' SELECT *
FROM  students
WHERE first_name LIKE 'M%'
'''

In [ ]:
# Pandas Syntax
df[df['first_name'].str.startswith('M')]

##### finding the students whose ages are between 17 and 30 years old

In [ ]:
# SQL Syntax
''' SELECT *
FROM  students
WHERE age BETWEEN 17 AND 30
'''

In [ ]:
# Pandas Syntax
df[(df.age >= 17) & (df.age <= 30)]

###### sorting the students by their grades in descending order

In [ ]:
# SQL Syntax
''' SELECT *
FROM  students
ORDER BY grade DESC
'''

In [ ]:
# Pandas Syntax
df.sort_values(by=df['grade'], ascending=False)

###### select all topics without duplication

In [ ]:
# SQL Syntax
''' SELECT DISTINCT topic
from students
'''

In [ ]:
# Pandas Syntax
df['topic'].value_counts()
# df[df['topic'].duplicated(keep='last')]

##### Creating a Database from Scratch (Optional)

In [ ]:
# creating a SQLite database
import sqlite3

# an sql-query to create a database
query = '''
        CREATE TABLE students
        (a VARCHAR(20), b VARCHAR(20),
        c REAL, d INTEGER
        );
    '''

# create a new connection
con = sqlite3.connect('data.sqlite')

# execute the qurey
con.execute(query)
con.commit()

# insert new data to the table
new_query = "INSERT INTO students VALUES('Mustafa', 'Othman', 33.5, 6)"

con.execute(new_query)
con.commit()

# listing the new data
cursor = con.execute('SELECT * FROM students')
rows = cursor.fetchall()
rows

# converting the results to a DataFrame
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

##### Getting Data from a Database

In [ ]:
# Load sqlalchemy's create_engine
from sqlalchemy import create_engine

# Create database engine to manage connections
engine = create_engine("sqlite:///data/data.db")

# Getting tables names
table_names = engine.table_names()
table_names

In [ ]:
# Load entire weather table by table name
weather = pd.read_sql("weather", engine)
weather.head()

##### Your First SQL Query

In [ ]:
# Load entire weather table with SQL
weather = pd.read_sql("SELECT * FROM weather", engine)
weather.head()

In [ ]:
# Write query to get records from Brooklyn
query = """
            SELECT *
            FROM hpd311calls
            WHERE borough = 'BROOKLYN';
        """

# Query the database
brooklyn_calls = pd.read_sql(query, engine)
brooklyn_calls.head()

In [ ]:
# Write query to get records about plumbing in the Bronx
and_query = """
                SELECT *
                FROM hpd311calls
                WHERE borough = 'BRONX' AND complaint_type = 'PLUMBING';
                """

# Get calls about plumbing issues in the Bronx
bx_plumbing_calls = pd.read_sql(and_query, engine)
bx_plumbing_calls.head()

##### Grouping & Aggregation

In [ ]:
# Write query to get plumbing call counts by borough
query = """
            SELECT borough, COUNT(*)
            FROM hpd311calls
            WHERE complaint_type = 'PLUMBING'
            GROUP BY borough;
            """

# Query the database and create a dataframe
plumbing_call_counts = pd.read_sql(query, engine)
plumbing_call_counts

##### Loading Multiple Tables with joins

In [ ]:
# Get call counts by borough and join in population and housing counts
query = """
            SELECT hpd311calls.borough,
            COUNT(*),
            boro_census.total_population,
            boro_census.housing_units
            FROM hpd311calls
            JOIN boro_census
            ON hpd311calls.borough = boro_census.borough
            GROUP BY hpd311calls.borough;
            """

# Query the database and create a dataframe
call_counts = pd.read_sql(query, engine)
call_counts

==========

## Case-study: Chinook Database

In [ ]:
from IPython.display import Image
Image("data/chinook-erd.png")

##### Importing Libraries & Database

In [ ]:
# Load sqlalchemy's create_engine
from sqlalchemy import create_engine

# Create database engine to manage connections
engine = create_engine("sqlite:///data/chinook.sqlite")

In [ ]:
# Getting tables names
table_names = engine.table_names()
table_names

In [ ]:
# Load entire 'Album' table
album = pd.read_sql("Album", engine)
album.head()

In [ ]:
# Another way to get the entire 'Album' table
pd.read_sql_table("Album", engine).head()

##### Provide a query showing Customers (just their full names, customer ID and country) who are not in the US.


In [ ]:
# Write query to get the specified customers' info
query = '''
select customerid, firstname, lastname, country
from customer
where not country = 'USA';
'''

In [ ]:
pd.read_sql(query, engine).head()

##### Provide a query only showing the Customers from Brazil.

In [ ]:
# Let's use 'pd.read_sql_query' method for simplicity
pd.read_sql_query("select * from customer where country = 'Brazil';", engine)

##### Provide a query showing the Invoices of customers who are from Brazil. The resultant table should show the customer's full name, Invoice ID, Date of the invoice and billing country.

In [ ]:
query = '''
select c.firstname, c.lastname, i.invoiceid, i.invoicedate, i.billingcountry
from customer as c, invoice as i
where c.country = 'Brazil' and
c.customerid = i.customerid;
'''
pd.read_sql(query, engine).head()


##### Provide a query showing only the Employees who are Sales Agents.

In [ ]:
pd.read_sql("select * from employee where employee.title = 'Sales Support Agent';", engine)

##### Provide a query showing a unique list of billing countries from the Invoice table.

In [ ]:
pd.read_sql_query("select distinct billingcountry from invoice;", engine)

##### Provide a query showing the invoices of customers who are from Brazil.

In [ ]:
query = '''
select *
from customer as c, invoice as i
where c.country = 'Brazil' and
c.customerid = i.customerid;
'''

pd.read_sql(query, engine).head()

##### Provide a query that shows the invoices associated with each sales agent. The resultant table should include the Sales Agent's full name.

In [ ]:
query = '''
select e.firstname, e.lastname, i.invoiceid, i.customerid, i.invoicedate, i.billingaddress, i.billingcountry, i.billingpostalcode, i.total
from customer as c, invoice as i
on c.customerid = i.customerid
join employee as e
on e.employeeid = c.supportrepid
order by e.employeeid;
'''

pd.read_sql(query, engine).head()

##### Provide a query that shows the Invoice Total, Customer name, Country and Sale Agent name for all invoices and customers.

In [ ]:
query = '''
select e.firstname as 'employee first', e.lastname as 'employee last', c.firstname as 'customer first', c.lastname as 'customer last', c.country, i.total
from employee as e
join customer as c on e.employeeid = c.supportrepid
join invoice as i on c.customerid = i.customerid
'''

pd.read_sql(query, engine).head()

##### How many Invoices were there in 2009 and 2011? What are the respective total sales for each of those years?

In [ ]:
query = '''
select count(i.invoiceid), sum(i.total)
from invoice as i
where i.invoicedate between datetime('2011-01-01 00:00:00') and datetime('2011-12-31 00:00:00');
'''

pd.read_sql(query, engine).head()

##### Looking at the InvoiceLine table, provide a query that COUNTs the number of line items for Invoice ID 37.

In [ ]:
pd.read_sql("select count(i.invoicelineid) from invoiceline as i where i.invoiceid = 37", engine)

==========

# THANK YOU!